Nuskaito knygų kategorijos **Literatūra mažiausiems** nuorodas ir išsaugoja puslapius

In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
url = "https://www.knygos.lt/lt/knygos/zanras/literatura-maziausiems/"
r = requests.get(url)
soup = BeautifulSoup(r.text)

psl_sk = int(soup.find(class_="numbers d-flex").find_all("li")[-2].text)
kategorija = urlparse(url).path.strip("/").split("/")[-1]
print(kategorija, psl_sk)

literatura-maziausiems 108


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random

with open("literatura_maziausiems_links.txt", "a+") as file:
    for page in range(1, psl_sk + 1):
        url = f"https://www.knygos.lt/lt/knygos/zanras/{kategorija}/?psl={page}"
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            book_elements = soup.find_all("a", class_="product-link")
            for book in book_elements:
                href = book.get("href").strip()
                file.write(href + "\n")
        else:
            print(f"Page {page} not accessible.")
        time.sleep(random.randint(1,10))

print("Links saved to literatura_maziausiems_links.txt")

 Sukuriamas aplankas saugojimui

In [2]:
import os

# Nurodomas aplanko pavadinimą
folder_name = "Literatura_maziausiems_html"

# Sukuriamas aplanką, jei jo dar nėra
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Aplankas '{folder_name}' buvo sukurtas.")
else:
    print(f"Aplankas '{folder_name}' jau egzistuoja.")

Aplankas 'Literatura_maziausiems_html' buvo sukurtas.


Nuskaitomos knygų nuorodos ir išsaugomos į tekstinį failą **literatura_maziausiems_links.txt**

Išsaugomi puslapiai į aplanką **Literatura_maziausiems_html**

In [ ]:
# Knygų puslapiai saugoti 2024-11-07
from urllib.parse import urlparse
import requests
import time

# Nuskaityti knygų nuorodas iš failo knygu_nuorodos.txt
with open("literatura_maziausiems_links.txt", "r") as file:
    books_links = [line.strip() for line in file.readlines() if line.strip()]

    for link in books_links:
        try:
            # Užkrauti knygos puslapį
            book_response = requests.get(link)
            path_parts = urlparse(link).path.strip("/").split("/")
            with open(f"{folder_name}/{path_parts[-1]}.html", "wb") as file:  #
                file.write(book_response.content)
            time.sleep(1)
        except Exception as e:
            print(f"Klaida renkant informaciją apie knygą {link}: {e}")
            time.sleep(5)  # Pristabdymas tarp užklausų
            continue

Saugoti knygų viršelius

In [3]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep
import random

# Nurodyti aplanką su knygų HTML failais ir aplanką, kur saugoti viršelius
input_folder = "Literatura_maziausiems_html"
output_folder = "Literatura_maziausiems_virseliai1"

# Patikrinti, ar išvesties aplankas egzistuoja, jei ne – sukurkite jį
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Eiti per kiekvieną failą aplanke
for file_name in tqdm(os.listdir(input_folder)):
    file_path = os.path.join(input_folder, file_name)

    # Atidaryti ir skaityti HTML failą
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    # Sukurti BeautifulSoup objektą, kad būtų galima analizuoti HTML
    soup = BeautifulSoup(html_content, "html.parser")

    # Surasti knygos pavadinimą ir viršelio nuorodą
    cover_img_tag = soup.find("img", itemprop="image")

    if cover_img_tag:
        cover_url = cover_img_tag["src"]
        file_extension = os.path.splitext(cover_url)[1]  # Ištraukti failo plėtinį iš viršelio URL

        # Atsisiųsti viršelio paveikslėlį
        response = requests.get(cover_url)
        if response.status_code == 200:
            # Išsaugoti viršelį su knygos pavadinimu ir tinkamu failo formatu
            file_title = file_name.split(".")[0]
            image_path = os.path.join(output_folder, f"{file_title}{file_extension}")

            with open(image_path, "wb") as img_file:
                img_file.write(response.content)

        else:
            print(f"Nepavyko atsisiųsti viršelio: {file_title}")
    else:
        print(f"Viršelio nuoroda nerasta: {file_title}")
    sleep(random.randint(1,10))

  9%|▉         | 300/3185 [37:32<6:01:02,  7.51s/it]


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))